## Q.11. 平滑化フィルタ

平滑化フィルタ(3x3)を実装せよ。

平滑化フィルタはフィルタ内の画素の平均値を出力するフィルタである。

In [3]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

def MeanFilter(image, kernel_size):
    # 画像がカラー画像か確認する。グレースケールの場合は次元を拡張する。
    if len(image.shape) == 3:
        H, W, C = image.shape
    else : # np.expand_dims()を用いると３次元に拡張される
        H, W, C = np.expand_dims(image, axis=-1)

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H + pad * 2, W + pad * 2, C), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad: pad + H, pad: pad + W] = image.copy().astype(float)

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
            for c in range(C):
                out[pad+y, pad+x, c] = np.mean(tmp[y:y+kernel_size, x:x+kernel_size, c])

    # 値を越えないようにクリップする
    out = np.clip(out, 0, 255)
    out = out[pad: pad + H, pad: pad + W].astype(np.uint8)
    return out

img_ = MeanFilter(img, 3)

# 保存して確認する
cv2.imwrite("training_IMG/training_11.png", img_)

True

In [ ]:
%reset -f

## Q.12. モーションフィルタ

モーションフィルタ(3x3)を実装せよ。

モーションフィルタとは対角方向の平均値を取るフィルタであり、次式で定義される。

```bash
  1/3  0   0
[  0  1/3  0 ]
   0   0  1/3
```

In [4]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

def MotionFilter(image, kernel_size):
    # 画像がカラー画像か確認する。グレースケールの場合は次元を拡張する。
    if len(image.shape) == 3:
        H, W, C = image.shape
    else : # np.expand_dims()を用いると３次元に拡張される
        H, W, C = np.expand_dims(image, axis=-1)

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H + pad * 2, W + pad * 2, C), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad: pad + H, pad: pad + W] = image.copy().astype(float)

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
            for c in range(C):
                out[pad+y, pad+x, c] = np.mean(np.diag(tmp[y:y+kernel_size, x:x+kernel_size, c]))

    # 値を越えないようにクリップする
    out = np.clip(out, 0, 255)
    out = out[pad: pad + H, pad: pad + W].astype(np.uint8)
    return out

img_ = MotionFilter(img, 3)

# 保存して確認する
cv2.imwrite("training_IMG/training_12.png", img_)

True

In [5]:
%reset -f

## Q.13. MAX-MINフィルタ

MAX-MINフィルタ(3x3)を実装せよ。

MAX-MINフィルタとはフィルタ内の画素の最大値と最小値の差を出力するフィルタであり、**エッジ検出**のフィルタの一つである。
エッジ検出とは画像内の線を検出るすることであり、このような画像内の情報を抜き出す操作を**特徴抽出**と呼ぶ。
エッジ検出では多くの場合、グレースケール画像に対してフィルタリングを行う。


In [6]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg").astype(np.float32)

def MaxMinFilter(image, kernel_size):
    # 画像がカラー画像か確認する。グレースケールの場合は次元を拡張する。
    if len(image.shape) == 3:
        H, W, C = image.shape
    else : # np.expand_dims()を用いると３次元に拡張される
        H, W, C = np.expand_dims(image, axis=-1)

    # パディングの大きさを決定する
    pad = kernel_size // 2
	# 上段下段、左右一列にパディングを入れたとして、その大きさの出力を用意する
    out = np.zeros((H + pad * 2, W + pad * 2, C), dtype=float)
	# パディングで囲われた内側にもとの画像を貼り付ける
    out[pad: pad + H, pad: pad + W] = image.copy().astype(float)

    # 作成したカーネルを用いてフィルタリングを行う
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
            for c in range(C):
                out[pad+y, pad+x, c] = np.max(tmp[y:y+kernel_size, x:x+kernel_size, c]) - np.min(tmp[y:y+kernel_size, x:x+kernel_size, c])

    # 値を越えないようにクリップする
    out = np.clip(out, 0, 255)
    out = out[pad: pad + H, pad: pad + W].astype(np.uint8)
    return out

img_ = MaxMinFilter(img, 3)

# 保存して確認する
cv2.imwrite("training_IMG/training_13.png", img_)

True

In [ ]:
%reset -f